In [ ]:
from u_cmab import Fourier
from u_cmab import Static_UM
from u_cmab import plot_results
import simulation_code

import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

from pylift import TransformedOutcome
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def restyle(ax, yticks):
    ax.title.set_fontsize(30)
    ax.title.set_text("")
    ax.title.set_fontfamily("sans-serif")
    
    ax.xaxis.label.set_fontsize(30)
    ax.xaxis.label.set_fontfamily("sans-serif")
    ax.xaxis.label.set_text('Fraction of data')

    ax.yaxis.label.set_fontsize(30)
    ax.yaxis.label.set_fontfamily("sans-serif")
    ax.yaxis.label.set_text('Uplift')

    ax.set_yticks(yticks)
    ax.set_xticks([0, .5, 1])

    ax.xaxis.set_tick_params(labelsize=30)
    ax.yaxis.set_tick_params(labelsize=30)

    ax.figure.set_size_inches(13.5, 10.5)

    font = fm.FontProperties(family='sans-serif', size=23)
  
    for l in ax.get_lines():
        if l.get_label() == "Random selection":
            l.set_linewidth(3)
            l.set_dashes((5,5))
            l.set_color('black')
            
    ax.legend(prop=font, loc='upper left')

    return ax

# Collect data
Sudden drift at exp. #10000

In [ ]:
sim = simulation_code.Simulation(D=2, std=0.2, drift_rate=3.5, sudden_drift=True, drift_moments=np.array([10000]), base_functions=np.array([simulation_code.Sine_Base(f=4), simulation_code.Sine_Base(f=5)]))

fourier = Fourier(sim, order=5)

epsilon = .1
alpha = .005

_ = fourier.run(epsilon, alpha, .2, 100, lifetime=20000)

# Train static uplift model with `pylift`
- `up_before`: training set `[0:7001]`, test set `[7001:20001]`
- `up_after`: training set `[10000:17001]`, test set `[17001:20001]`
- `up_before`: training set `[0:17001]`, test set `[17001:20001]`

In [ ]:
#URF
up_before = TransformedOutcome((
    fourier.RP_run_history.iloc[:,0:4][0:7001], 
    fourier.RP_run_history.iloc[:,0:4][7001:10001]), 
    col_treatment='C', col_outcome='E', 
    stratify=None, 
    sklearn_model=RandomForestRegressor)
up_after = TransformedOutcome((
    fourier.RP_run_history.iloc[:,0:4][10000:17001], 
    fourier.RP_run_history.iloc[:,0:4][17001:20001]), 
    col_treatment='C', col_outcome='E', 
    stratify=None, 
    sklearn_model=RandomForestRegressor)
up_across = TransformedOutcome((
    fourier.RP_run_history.iloc[:,0:4][0:17001], 
    fourier.RP_run_history.iloc[:,0:4][17001:20001]), 
    col_treatment='C', col_outcome='E', 
    stratify=None, 
    sklearn_model=RandomForestRegressor)

In [ ]:
#XGB
up_before_xgb = TransformedOutcome((
    fourier.RP_run_history.iloc[:,0:4][0:7001], 
    fourier.RP_run_history.iloc[:,0:4][7001:10001]), 
    col_treatment='C', col_outcome='E', 
    stratify=None)
up_after_xgb = TransformedOutcome((
    fourier.RP_run_history.iloc[:,0:4][10000:17001], 
    fourier.RP_run_history.iloc[:,0:4][17001:20001]), 
    col_treatment='C', col_outcome='E', 
    stratify=None)
up_across_xgb = TransformedOutcome((
    fourier.RP_run_history.iloc[:,0:4][0:17001], 
    fourier.RP_run_history.iloc[:,0:4][17001:20001]), 
    col_treatment='C', col_outcome='E', 
    stratify=None)

In [ ]:
up_before.randomized_search(
    param_distributions={'max_depth': range(2,100), 'min_samples_split': range(2,1000)}, 
    n_iter=50, n_jobs=10)
up_after.randomized_search(
    param_distributions={'max_depth': range(2,100), 'min_samples_split': range(2,1000)}, 
    n_iter=50, n_jobs=10)
up_across.randomized_search(
    param_distributions={'max_depth': range(2,100), 'min_samples_split': range(2,1000)}, 
    n_iter=50, n_jobs=10)

In [ ]:
up_before_xgb.randomized_search()
up_after_xgb.randomized_search()
up_across_xgb.randomized_search()

In [ ]:
up_before.fit(**up_before.rand_search_.best_params_)
up_after.fit(**up_after.rand_search_.best_params_)
up_across.fit(**up_across.rand_search_.best_params_)

In [ ]:
up_before_xgb.fit(**up_before_xgb.rand_search_.best_params_)
up_after_xgb.fit(**up_after_xgb.rand_search_.best_params_)
up_across_xgb.fit(**up_across_xgb.rand_search_.best_params_)

### Noise fits

In [ ]:
up_before.noise_fit(iterations=25)
up_after.noise_fit(iterations=25)
up_across.noise_fit(iterations=25)

In [ ]:
up_before_xgb.noise_fit(iterations=25)
up_after_xgb.noise_fit(iterations=25)
up_across_xgb.noise_fit(iterations=25)

# Plot results

In [ ]:
ax1 = restyle(up_before.plot(show_noise_fits=True,label="URF", color='firebrick', markersize=13, linewidth=3), [0, .14])
ax2 = restyle(up_after.plot(show_noise_fits=True,label="URF", color='firebrick', markersize=13, linewidth=3), [0, .13])
ax3 = restyle(up_across.plot(show_noise_fits=True,label="URF", color='firebrick', markersize=13, linewidth=3), [-.01, .02])

In [ ]:
axxb1 = restyle(up_before_xgb.plot(show_noise_fits=True,label="XGB", color='mediumvioletred', markersize=13, linewidth=3), [0, 0.1])
axxb2 = restyle(up_after_xgb.plot(show_noise_fits=True,label="XGB", color='mediumvioletred', markersize=13, linewidth=3), [-.01, .04])
axxb3 = restyle(up_across_xgb.plot(show_noise_fits=True,label="XGB", color='mediumvioletred', markersize=13, linewidth=3), [-.01, .03])

### Save plots